In [ ]:
#!/usr/bin/env python3
# ===============================================
# PP_16 - TABNET + NEURAL META-LEARNER (GOD SOTA 2026)
# TennisTitan - Advanced Stacking Architecture
# ===============================================
#
# Ajouts:
# 1. TabNet comme 4ème modèle de base (attention intégrée)
# 2. Neural Meta-Learner (MLP) au lieu de LogReg
# 3. Multi-seed + OOF stacking
# 4. Calibration avancée (Isotonic + Temperature)
#
# Input: data_clean/ml_final/
# Output: models/god_sota_2026/
# ===============================================

import numpy as np
import polars as pl
import pandas as pd
from pathlib import Path
from datetime import datetime
import json
import joblib
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, roc_auc_score, brier_score_loss
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

try:
    from venn_abers import VennAbersCalibrator
    VENN_ABERS_AVAILABLE = True
except ImportError:
    print("⚠️ Venn-ABERS not installed. Run: pip install venn-abers")
    VENN_ABERS_AVAILABLE = False
    
# TabNet
try:
    from pytorch_tabnet.tab_model import TabNetClassifier
    TABNET_AVAILABLE = True
except ImportError:
    print("⚠️ TabNet not installed. Run: pip install pytorch-tabnet")
    TABNET_AVAILABLE = False

# ===============================================
# CONFIGURATION
# ===============================================
ROOT = Path(r"C:\Users\Administrateur\Tennis POLAR v2")
DATA_DIR = ROOT / "data_clean" / "ml_final"
OUTPUT_DIR = ROOT / "models" / "god_sota_2026"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Seeds for multi-seed ensembling
SEEDS = [42, 123, 456, 789, 2024]
N_FOLDS = 5

# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("=" * 70)
print("   PP_16 - TABNET + NEURAL META-LEARNER (GOD SOTA 2026)")
print("=" * 70)
print(f"   {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"   Device: {DEVICE}")
print(f"   Seeds: {SEEDS}")
print(f"   TabNet available: {TABNET_AVAILABLE}")
print("=" * 70)


# ===============================================
# NEURAL META-LEARNER
# ===============================================

class NeuralMetaLearner(nn.Module):
    """
    MLP meta-learner to combine base model predictions.
    
    Input: [lgbm_prob, xgb_prob, cat_prob, tabnet_prob, context_features]
    Output: Final probability
    """
    
    def __init__(self, n_base_models: int = 4, n_context: int = 5, 
                 hidden_dim: int = 32, dropout: float = 0.3):
        super().__init__()
        
        input_dim = n_base_models + n_context
        
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout / 2),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
        
        # Label smoothing parameter
        self.label_smoothing = 0.05
    
    def forward(self, x):
        return self.network(x).squeeze(-1)
    
    def loss(self, preds, targets):
        """BCE with label smoothing."""
        # Smooth labels
        targets_smooth = targets * (1 - self.label_smoothing) + 0.5 * self.label_smoothing
        return F.binary_cross_entropy(preds, targets_smooth)


def train_neural_meta(X_meta: np.ndarray, y: np.ndarray, 
                      X_val: np.ndarray = None, y_val: np.ndarray = None,
                      epochs: int = 100, lr: float = 0.001, patience: int = 10):
    """Train the neural meta-learner."""
    
    model = NeuralMetaLearner(
        n_base_models=X_meta.shape[1] - 5 if X_meta.shape[1] > 5 else X_meta.shape[1],
        n_context=min(5, X_meta.shape[1]),
        hidden_dim=32,
        dropout=0.3
    ).to(DEVICE)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )
    
    # Prepare data
    X_train_t = torch.tensor(X_meta, dtype=torch.float32).to(DEVICE)
    y_train_t = torch.tensor(y, dtype=torch.float32).to(DEVICE)
    
    if X_val is not None:
        X_val_t = torch.tensor(X_val, dtype=torch.float32).to(DEVICE)
        y_val_t = torch.tensor(y_val, dtype=torch.float32).to(DEVICE)
    
    best_loss = float('inf')
    patience_counter = 0
    best_state = None
    
    for epoch in range(epochs):
        # Training
        model.train()
        preds = model(X_train_t)
        loss = model.loss(preds, y_train_t)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Validation
        if X_val is not None:
            model.eval()
            with torch.no_grad():
                val_preds = model(X_val_t)
                val_loss = F.binary_cross_entropy(val_preds, y_val_t)
            
            scheduler.step(val_loss)
            
            if val_loss < best_loss:
                best_loss = val_loss.item()
                patience_counter = 0
                best_state = model.state_dict()
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    break
    
    if best_state:
        model.load_state_dict(best_state)
    
    return model


# ===============================================
# TABNET WRAPPER
# ===============================================

def train_tabnet(X_train, y_train, X_val, y_val, seed=42):
    """Train TabNet classifier."""
    
    if not TABNET_AVAILABLE:
        return None
    
    model = TabNetClassifier(
        n_d=64,
        n_a=64,
        n_steps=5,
        gamma=1.5,
        lambda_sparse=0.001,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2),
        scheduler_params={"step_size": 10, "gamma": 0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='entmax',
        seed=seed,
        verbose=0,
        device_name='cuda' if torch.cuda.is_available() else 'cpu',
    )
    
    model.fit(
        X_train=X_train,
        y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_metric=['logloss'],
        max_epochs=100,
        patience=15,
        batch_size=1024,
        virtual_batch_size=256,
    )
    
    return model


# ===============================================
# OOF STACKING
# ===============================================

def compute_oof_predictions(X, y, model_type: str, seeds: list, n_folds: int = 5):
    """
    Compute Out-of-Fold predictions for a model.
    Returns OOF predictions and trained models.
    """
    
    oof_preds = np.zeros(len(y))
    models = []
    
    for seed in seeds:
        kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
        seed_models = []
        
        for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            if model_type == "lgbm":
                model = lgb.LGBMClassifier(
                    n_estimators=1000,
                    learning_rate=0.05,
                    num_leaves=31,
                    max_depth=8,
                    min_child_samples=100,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    reg_alpha=0.1,
                    reg_lambda=1.0,
                    random_state=seed,
                    verbosity=-1,
                    force_row_wise=True,
                )
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_val, y_val)],
                    callbacks=[lgb.early_stopping(100, verbose=False)]
                )
                preds = model.predict_proba(X_val)[:, 1]
                
            elif model_type == "xgb":
                model = xgb.XGBClassifier(
                    n_estimators=1000,
                    learning_rate=0.05,
                    max_depth=6,
                    min_child_weight=10,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    reg_alpha=0.1,
                    reg_lambda=1.0,
                    random_state=seed,
                    eval_metric='logloss',
                    early_stopping_rounds=100,
                    verbosity=0,
                )
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
                preds = model.predict_proba(X_val)[:, 1]
                
            elif model_type == "cat":
                model = CatBoostClassifier(
                    iterations=1000,
                    learning_rate=0.05,
                    depth=6,
                    l2_leaf_reg=3,
                    random_seed=seed,
                    verbose=False,
                    early_stopping_rounds=100,
                )
                model.fit(X_train, y_train, eval_set=(X_val, y_val))
                preds = model.predict_proba(X_val)[:, 1]
                
            elif model_type == "tabnet" and TABNET_AVAILABLE:
                model = train_tabnet(X_train, y_train, X_val, y_val, seed)
                if model is not None:
                    preds = model.predict_proba(X_val)[:, 1]
                else:
                    preds = np.full(len(val_idx), 0.5)
            else:
                continue
            
            oof_preds[val_idx] += preds / len(seeds)
            seed_models.append(model)
        
        models.append(seed_models)
    
    return oof_preds, models


# ===============================================
# CALIBRATION
# ===============================================

class TemperatureScaling:
    """Temperature scaling for calibration."""
    
    def __init__(self):
        self.temperature = 1.0
    
    def fit(self, logits, y):
        """Find optimal temperature."""
        from scipy.optimize import minimize_scalar
        
        def nll(T):
            scaled = 1 / (1 + np.exp(-logits / T))
            scaled = np.clip(scaled, 1e-7, 1-1e-7)
            return log_loss(y, scaled)
        
        result = minimize_scalar(nll, bounds=(0.1, 10), method='bounded')
        self.temperature = result.x
        return self
    
    def transform(self, logits):
        return 1 / (1 + np.exp(-logits / self.temperature))


def calibrate_predictions(preds, y, method='isotonic'):
    """Calibrate predictions."""
    
    if method == 'isotonic':
        calibrator = IsotonicRegression(out_of_bounds='clip')
        calibrator.fit(preds, y)
        return calibrator
    
    elif method == 'platt':
        calibrator = LogisticRegression(C=1.0, solver='lbfgs')
        calibrator.fit(preds.reshape(-1, 1), y)
        return calibrator
    
    elif method == 'temperature':
        # Convert to logits
        logits = np.log(preds / (1 - preds + 1e-7))
        calibrator = TemperatureScaling()
        calibrator.fit(logits, y)
        return calibrator
    
    return None

def calibrate_venn_abers(train_probs: np.ndarray, train_labels: np.ndarray,
                         val_probs: np.ndarray = None):
    """
    Calibration Venn-ABERS - produit des intervalles de confiance.
    
    Returns:
        calibrator: Fitted VennAbersCalibrator
        p_low, p_high: Lower and upper probability bounds (if val_probs provided)
    """
    
    if not VENN_ABERS_AVAILABLE:
        print("  ⚠️ Venn-ABERS not available, skipping")
        return None, None, None
    
    calibrator = VennAbersCalibrator()
    
    # Venn-ABERS needs probabilities in shape (n_samples,)
    calibrator.fit(train_probs.reshape(-1), train_labels.reshape(-1))
    
    if val_probs is not None:
        # predict_proba returns (p_low, p_high) for each sample
        p_low, p_high = calibrator.predict_proba(val_probs.reshape(-1))
        return calibrator, p_low, p_high
    
    return calibrator, None, None
    
# ===============================================
# MAIN TRAINING PIPELINE
# ===============================================

def main():
    t0 = datetime.now()
    
    # Load data
    print("\n[1/7] Loading data...")
    
    train = pl.read_parquet(DATA_DIR / "train.parquet").to_pandas()
    val = pl.read_parquet(DATA_DIR / "val.parquet").to_pandas()
    test = pl.read_parquet(DATA_DIR / "test.parquet").to_pandas()
    
    # Prepare features
    exclude_cols = ["target_A_wins", "custom_match_id", "match_id_ta_dedup", 
                    "winner_id", "loser_id", "year"]
    feature_cols = [c for c in train.columns if c not in exclude_cols]
    
    X_train = train[feature_cols].values.astype(np.float32)
    y_train = train["target_A_wins"].values.astype(np.int32)
    X_val = val[feature_cols].values.astype(np.float32)
    y_val = val["target_A_wins"].values.astype(np.int32)
    X_test = test[feature_cols].values.astype(np.float32)
    y_test = test["target_A_wins"].values.astype(np.int32)
    
    # Replace NaN
    X_train = np.nan_to_num(X_train, nan=0.0)
    X_val = np.nan_to_num(X_val, nan=0.0)
    X_test = np.nan_to_num(X_test, nan=0.0)
    
    print(f"  Train: {X_train.shape}")
    print(f"  Val: {X_val.shape}")
    print(f"  Test: {X_test.shape}")
    print(f"  Features: {len(feature_cols)}")
    
    # Combine train + val for OOF
    X_trainval = np.vstack([X_train, X_val])
    y_trainval = np.concatenate([y_train, y_val])
    
    # =====================================
    # OOF PREDICTIONS
    # =====================================
    print("\n[2/7] Computing OOF predictions...")
    
    oof_results = {}
    all_models = {}
    
    for model_type in ["lgbm", "xgb", "cat", "tabnet"]:
        print(f"\n  Training {model_type.upper()}...")
        
        if model_type == "tabnet" and not TABNET_AVAILABLE:
            print(f"  ⚠️ Skipping TabNet (not installed)")
            continue
        
        oof_preds, models = compute_oof_predictions(
            X_trainval, y_trainval, model_type, SEEDS, N_FOLDS
        )
        
        oof_results[model_type] = oof_preds
        all_models[model_type] = models
        
        auc = roc_auc_score(y_trainval, oof_preds)
        ll = log_loss(y_trainval, np.clip(oof_preds, 1e-7, 1-1e-7))
        print(f"  {model_type.upper()} OOF: AUC={auc:.4f}, LogLoss={ll:.4f}")
    
    # =====================================
    # TRAIN META-LEARNER ON OOF
    # =====================================
    print("\n[3/7] Training Neural Meta-Learner...")
    
    # Create meta features
    meta_cols = list(oof_results.keys())
    X_meta_trainval = np.column_stack([oof_results[k] for k in meta_cols])
    
    # Add context features (rank diff, surface one-hot, etc.)
    # For simplicity, we'll use base predictions only
    
    # Split for meta training
    meta_split = int(len(X_meta_trainval) * 0.9)
    X_meta_train = X_meta_trainval[:meta_split]
    y_meta_train = y_trainval[:meta_split]
    X_meta_val = X_meta_trainval[meta_split:]
    y_meta_val = y_trainval[meta_split:]
    
    neural_meta = train_neural_meta(
        X_meta_train, y_meta_train,
        X_meta_val, y_meta_val,
        epochs=100, lr=0.001, patience=10
    )
    
    # OOF prediction with neural meta
    neural_meta.eval()
    with torch.no_grad():
        X_meta_t = torch.tensor(X_meta_trainval, dtype=torch.float32).to(DEVICE)
        oof_meta = neural_meta(X_meta_t).cpu().numpy()
    
    meta_auc = roc_auc_score(y_trainval, oof_meta)
    meta_ll = log_loss(y_trainval, np.clip(oof_meta, 1e-7, 1-1e-7))
    print(f"  Neural Meta OOF: AUC={meta_auc:.4f}, LogLoss={meta_ll:.4f}")
    
    # =====================================
    # TRAIN FINAL MODELS ON ALL DATA
    # =====================================
    print("\n[4/7] Training final models on full train+val...")
    
    final_models = {}
    
    for model_type in meta_cols:
        print(f"  Training final {model_type.upper()}...")
        
        if model_type == "lgbm":
            model = lgb.LGBMClassifier(
                n_estimators=1000, learning_rate=0.05, num_leaves=31,
                max_depth=8, min_child_samples=100, subsample=0.8,
                colsample_bytree=0.8, reg_alpha=0.1, reg_lambda=1.0,
                random_state=42, verbosity=-1
            )
            model.fit(X_trainval, y_trainval)
            
        elif model_type == "xgb":
            model = xgb.XGBClassifier(
                n_estimators=1000, learning_rate=0.05, max_depth=6,
                min_child_weight=10, subsample=0.8, colsample_bytree=0.8,
                reg_alpha=0.1, reg_lambda=1.0, random_state=42, verbosity=0
            )
            model.fit(X_trainval, y_trainval)
            
        elif model_type == "cat":
            model = CatBoostClassifier(
                iterations=1000, learning_rate=0.05, depth=6,
                l2_leaf_reg=3, random_seed=42, verbose=False
            )
            model.fit(X_trainval, y_trainval)
            
        elif model_type == "tabnet" and TABNET_AVAILABLE:
            model = train_tabnet(X_trainval, y_trainval, X_val, y_val, seed=42)
        
        final_models[model_type] = model
    
    # =====================================
    # TEST SET PREDICTIONS
    # =====================================
    print("\n[5/7] Generating test predictions...")
    
    test_preds = {}
    for model_type, model in final_models.items():
        if model is not None:
            test_preds[model_type] = model.predict_proba(X_test)[:, 1]
            print(f"  {model_type.upper()}: AUC={roc_auc_score(y_test, test_preds[model_type]):.4f}")
    
    # Meta prediction on test
    X_meta_test = np.column_stack([test_preds[k] for k in meta_cols if k in test_preds])
    
    neural_meta.eval()
    with torch.no_grad():
        X_test_t = torch.tensor(X_meta_test, dtype=torch.float32).to(DEVICE)
        test_meta = neural_meta(X_test_t).cpu().numpy()
    
    print(f"\n  Neural Meta Test: AUC={roc_auc_score(y_test, test_meta):.4f}")
    
    # =====================================
    # CALIBRATION
    # =====================================
    print("\n[6/7] Calibrating predictions...")
    
    # Isotonic on validation predictions
    val_preds = {}
    for model_type, model in final_models.items():
        if model is not None:
            val_preds[model_type] = model.predict_proba(X_val)[:, 1]
    
    X_meta_val_final = np.column_stack([val_preds[k] for k in meta_cols if k in val_preds])
    
    with torch.no_grad():
        X_val_t = torch.tensor(X_meta_val_final, dtype=torch.float32).to(DEVICE)
        val_meta = neural_meta(X_val_t).cpu().numpy()
    
    # Fit calibrators
    isotonic = calibrate_predictions(val_meta, y_val, method='isotonic')
    platt = calibrate_predictions(val_meta, y_val, method='platt')
    
    # Apply calibration
    test_calibrated_iso = isotonic.transform(test_meta)
    test_calibrated_platt = platt.predict_proba(test_meta.reshape(-1, 1))[:, 1]
    
    print(f"  Isotonic Test: AUC={roc_auc_score(y_test, test_calibrated_iso):.4f}, "
          f"Brier={brier_score_loss(y_test, test_calibrated_iso):.4f}")
    print(f"  Platt Test: AUC={roc_auc_score(y_test, test_calibrated_platt):.4f}, "
          f"Brier={brier_score_loss(y_test, test_calibrated_platt):.4f}")

    if VENN_ABERS_AVAILABLE:
        print("\n  Fitting Venn-ABERS calibration...")
        
        # Fit on validation set
        venn_abers, _, _ = calibrate_venn_abers(val_meta, y_val)
        
        if venn_abers is not None:
            # Apply to test set
            p_low, p_high = venn_abers.predict_proba(test_meta.reshape(-1))
            
            # Point estimate (average of bounds)
            test_calibrated_venn = (p_low + p_high) / 2
            
            # Uncertainty (width of interval)
            test_uncertainty = p_high - p_low
            
            # Metrics
            venn_auc = roc_auc_score(y_test, test_calibrated_venn)
            venn_brier = brier_score_loss(y_test, test_calibrated_venn)
            
            print(f"  Venn-ABERS Test: AUC={venn_auc:.4f}, Brier={venn_brier:.4f}")
            print(f"  Mean uncertainty: {test_uncertainty.mean():.4f}")
            
            # Compare calibrations
            print("\n  📊 Calibration Comparison:")
            print(f"     Isotonic:   Brier={brier_score_loss(y_test, test_calibrated_iso):.4f}")
            print(f"     Platt:      Brier={brier_score_loss(y_test, test_calibrated_platt):.4f}")
            print(f"     Venn-ABERS: Brier={venn_brier:.4f}")
    # =====================================
    # SAVE EVERYTHING
    # =====================================
    print("\n[7/7] Saving models and results...")
    
    # Final predictions
    final_preds = test_calibrated_iso
    final_auc = roc_auc_score(y_test, final_preds)
    final_ll = log_loss(y_test, np.clip(final_preds, 1e-7, 1-1e-7))
    final_brier = brier_score_loss(y_test, final_preds)
    
    # Save models
    joblib.dump({
        'lgbm': final_models.get('lgbm'),
        'xgb': final_models.get('xgb'),
        'cat': final_models.get('cat'),
        'tabnet': final_models.get('tabnet'),
        'isotonic': isotonic,
        'platt': platt,
        'venn_abers': venn_abers if VENN_ABERS_AVAILABLE else None, 
        'feature_cols': feature_cols,
    }, OUTPUT_DIR / "final_models.joblib")
    
    # Save neural meta
    torch.save(neural_meta.state_dict(), OUTPUT_DIR / "neural_meta.pt")
    
    # Save metrics
    metrics = {
        'test_auc': final_auc,
        'test_logloss': final_ll,
        'test_brier': final_brier,
        'oof_auc': meta_auc,
        'oof_logloss': meta_ll,
        'models': list(meta_cols),
        'created': datetime.now().isoformat(),
    }
    
    with open(OUTPUT_DIR / "metrics.json", "w") as f:
        json.dump(metrics, f, indent=2)
    
    elapsed = (datetime.now() - t0).total_seconds()
    
    print("\n" + "=" * 70)
    print("   🏆 FINAL RESULTS (TEST SET)")
    print("=" * 70)
    print(f"\n   AUC:     {final_auc:.4f}")
    print(f"   LogLoss: {final_ll:.4f}")
    print(f"   Brier:   {final_brier:.4f}")
    print(f"\n   ⏱️ Time: {elapsed/60:.1f} minutes")
    print(f"   📁 Output: {OUTPUT_DIR}")
    
    print("\n" + "=" * 70)
    print("   ✅ PP_16 GOD SOTA COMPLETE!")
    print("=" * 70)
    print("""
📋 COMPOSANTS SAUVÉS:
   • final_models.joblib: LGBM, XGB, CatBoost, TabNet
   • neural_meta.pt: Neural Meta-Learner
   • metrics.json: Résultats finaux

🎯 ARCHITECTURE:
   Input Features (198)
        ↓
   [LGBM] [XGB] [CatBoost] [TabNet*]
        ↓        ↓           ↓         ↓
        └────────┴───────────┴─────────┘
                       ↓
              Neural Meta-Learner (MLP)
                       ↓
              Isotonic Calibration
                       ↓
              Final Probability

* TabNet optionnel (nécessite pytorch-tabnet)
""")
    
    return metrics

def select_best_calibration(test_preds: np.ndarray, y_test: np.ndarray,
                            isotonic_preds: np.ndarray,
                            platt_preds: np.ndarray,
                            venn_preds: np.ndarray = None) -> tuple:
    """
    Sélectionne automatiquement la meilleure méthode de calibration
    basée sur le Brier Score.
    
    Returns:
        best_preds: Array of best calibrated predictions
        best_method: Name of best method
    """
    
    candidates = {
        'isotonic': isotonic_preds,
        'platt': platt_preds,
    }
    
    if venn_preds is not None:
        candidates['venn_abers'] = venn_preds
    
    brier_scores = {}
    for name, preds in candidates.items():
        brier_scores[name] = brier_score_loss(y_test, preds)
    
    best_method = min(brier_scores, key=brier_scores.get)
    best_preds = candidates[best_method]
    
    print(f"\n  🏆 Best calibration: {best_method} (Brier={brier_scores[best_method]:.4f})")
    
    return best_preds, best_method

def get_confidence_adjusted_prediction(p_low: np.ndarray, p_high: np.ndarray,
                                       confidence_threshold: float = 0.15) -> tuple:
    """
    Retourne les prédictions avec un flag de confiance.
    
    Args:
        p_low: Lower probability bounds
        p_high: Upper probability bounds
        confidence_threshold: Max uncertainty pour être "confident"
    
    Returns:
        predictions: Point estimates
        is_confident: Boolean mask for high-confidence predictions
    """
    
    predictions = (p_low + p_high) / 2
    uncertainty = p_high - p_low
    is_confident = uncertainty <= confidence_threshold
    
    n_confident = is_confident.sum()
    pct_confident = 100 * n_confident / len(is_confident)
    
    print(f"  Confident predictions: {n_confident:,} ({pct_confident:.1f}%)")
    print(f"  Mean uncertainty (confident): {uncertainty[is_confident].mean():.4f}")
    print(f"  Mean uncertainty (uncertain): {uncertainty[~is_confident].mean():.4f}")
    
    return predictions, is_confident
    
if __name__ == "__main__":
    metrics = main()
